# Accessing OpenBIDS for CML Data
This tutorial shows how to load behavioral and EEG data in OpenBIDS format so it matches the structure used by the Computational Memory Lab’s CMLReader package. The structure is designed to support new analyses and replicate existing scripts with minimal changes.

In [1]:
# imports
import pandas as pd; pd.set_option('display.max_columns', None)
import cmlreaders as cml
import numpy as np
import xarray as xr
from mne_bids import BIDSPath, read_raw_bids, get_entity_vals
import os
import mne
from ptsa.data.timeseries import TimeSeries

## Loading Behavioral Data

### CMLReader

As a reminder, to load behavioral data for a given experimental session, we select the session using its subject, experiment, and session identifiers, instantiate a CMLReader object, and then load the desired data type (e.g., events). 

In [2]:
# load dataframe of all sessions
df = cml.get_data_index()
df[:10]                     # show the first 10 entries

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
0,NaN,protocols/ltp/subjects/LTP001/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,0,NaN,NaN,0,LTP001,LTP001,NaN,protocols/ltp/subjects/LTP001/experiments/Valu...
1,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,0,NaN,NaN,0,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
2,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,1,NaN,NaN,1,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
3,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,10,NaN,NaN,10,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
4,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,11,NaN,NaN,11,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
5,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,12,NaN,NaN,12,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
6,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,13,NaN,NaN,13,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
7,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,14,NaN,NaN,14,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
8,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,15,NaN,NaN,15,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
9,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,16,NaN,NaN,16,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...


In [30]:
df_PS2 = df.query("experiment == 'PS2'").iloc[0]

In [31]:
cmlreader = cml.CMLReader(subject=df_PS2['subject'], experiment=df_PS2['experiment'], session=df_PS2['session'])
# load the behavioral events
evs_PS2 = cmlreader.load('events')

In [32]:
evs_PS2[evs_PS2['type']== 'STIM_ON']

,eegoffset,ad_observed,eegfile,exp_version,experiment,is_stim,montage,msoffset,mstime,protocol,session,stim_params,subject,type
1,57184,False,R1050M_PS2_0_04Jun15_2008,1,PS2,True,0,3,1433463199591,r1,0,"[{'amplitude': 1000.0, 'anode_label': 'LPG42',...",R1050M,STIM_ON
3,58938,False,R1050M_PS2_0_04Jun15_2008,1,PS2,True,0,3,1433463203100,r1,0,"[{'amplitude': 1000.0, 'anode_label': 'LPG42',...",R1050M,STIM_ON
5,60679,False,R1050M_PS2_0_04Jun15_2008,1,PS2,True,0,3,1433463206582,r1,0,"[{'amplitude': 1000.0, 'anode_label': 'LPG42',...",R1050M,STIM_ON
7,62576,False,R1050M_PS2_0_04Jun15_2008,1,PS2,True,0,3,1433463210376,r1,0,"[{'amplitude': 1000.0, 'anode_label': 'LPG42',...",R1050M,STIM_ON
10,66110,False,R1050M_PS2_0_04Jun15_2008,1,PS2,True,0,3,1433463217445,r1,0,"[{'amplitude': 1000.0, 'anode_label': 'LPG42',...",R1050M,STIM_ON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,724648,False,R1050M_PS2_0_04Jun15_2008,1,PS2,True,0,3,1433464534593,r1,0,"[{'amplitude': 1500.0, 'anode_label': 'LPG42',...",R1050M,STIM_ON
657,726542,False,R1050M_PS2_0_04Jun15_2008,1,PS2,True,0,2,1433464538382,r1,0,"[{'amplitude': 1500.0, 'anode_label': 'LPG42',...",R1050M,STIM_ON
660,730280,False,R1050M_PS2_0_04Jun15_2008,1,PS2,True,0,3,1433464545858,r1,0,"[{'amplitude': 1500.0, 'anode_label': 'LPG42',...",R1050M,STIM_ON
662,732121,False,R1050M_PS2_0_04Jun15_2008,1,PS2,True,0,3,1433464549539,r1,0,"[{'amplitude': 1500.0, 'anode_label': 'LPG42',...",R1050M,STIM_ON


In [33]:
evs_PS2.columns

Index(['eegoffset', 'ad_observed', 'eegfile', 'exp_version', 'experiment',
       'is_stim', 'montage', 'msoffset', 'mstime', 'protocol', 'session',
       'stim_params', 'subject', 'type'],
      dtype='object')

In [34]:
evs_PS2["type"].unique()

array(['BEGIN_PS2', 'STIM_ON', 'STIM_OFF', 'STIM_SINGLE_PULSE',
       'AD_CHECK'], dtype=object)

In [3]:
# let's find subjects who did the ValueCourier experiment
VC_df = df.query("experiment == 'ValueCourier'")
VC_df['subject'].unique()[:20]

array(['LTP001', 'LTP606', 'LTP607', 'LTP609', 'LTP610', 'LTP612',
       'LTP613', 'LTP614', 'LTP9992', 'LTP9993'], dtype=object)

In [4]:
# we'll pick LTP606 and select out this subject's ValueCourier sessions
sub_scalp = 'LTP606'
exp_scalp = 'ValueCourier'
df_select = df[(df['subject'] == sub_scalp) & (df['experiment'] == exp_scalp)]
display(df_select); print(f'{sub_scalp} sessions: {np.array(df_select.session)}')

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
7696,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,0,NaN,NaN,0,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7697,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,1,NaN,NaN,1,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7698,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,2,NaN,NaN,2,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7699,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,3,NaN,NaN,3,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7700,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,4,NaN,NaN,4,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7701,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,5,NaN,NaN,5,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...


LTP606 sessions: [0 1 2 3 4 5]


In [5]:
# lets load the data from the first session
df_sess = df_select.iloc[0]         # select 1 row

# instantiate a Reader object using session metadata
# subjects beginning with 'LTP' are scalp subjects, so we don't need to specify the localization and montage
cmlreader = cml.CMLReader(subject=df_sess['subject'], experiment=df_sess['experiment'], session=df_sess['session'])
# load the behavioral events
evs_cml = cmlreader.load('events')
evs_cml[:10]

,eegoffset,actualvalue,compensation,correctPointingDirection,eegfile,eogArtifact,experiment,finalrecalled,intruded,intrusion,item,itemno,itemvalue,montage,msoffset,mstime,multiplier,numingroupchosen,phase,playerrotY,presX,presZ,primacybuf,protocol,recalled,recencybuf,rectime,serialpos,session,store,storeX,storeZ,storepointtype,subject,submittedPointingDirection,trial,type,valuerecall
0,23440,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761665861795,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,store mappings,-999
1,31922,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-1,-999,0,-1,1761665865937,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,VIDEO_START,-999
2,468068,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-1,-999,0,-1,1761666078916,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,VIDEO_STOP,-999
3,646089,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666165847,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999
4,646183,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666165893,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,TL_START,-999
5,1724585,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666692498,0.846154,4,1,-999.0,-3.000000,8.757812,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,TL_END,-999
6,1725717,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666693051,0.846154,4,1,-999.0,-3.937500,11.757812,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999
7,1731210,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666695733,0.846154,4,1,-999.0,-3.953125,11.804688,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999
8,1763086,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666711299,0.846154,4,1,-999.0,-3.953125,11.804688,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999
9,1766295,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666712866,0.846154,4,1,-999.0,-3.953125,11.804688,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999


### OpenBIDS

To load behavioral data in OpenBIDS, we first set the root of the bids database. The database is structured as such:

<pre>
root/
├── subjects/
│   └── sub-{subject_id}/
│       └── ses-{session_id}/
│           ├── beh/
│           │   ├── sub-{subject_id}_ses-{session_id}_task-{experiment}_beh.json
│           │   └── sub-{subject_id}_ses-{session_id}_task-{experiment}_beh.tsv
│           └── eeg/
│               ├── sub-{subject_id}_ses-{session_id}_task-{experiment}_eeg.bdf
│               └── sub-{subject_id}_ses-{session_id}_task-{experiment}_events.tsv
└── participants.tsv
</pre>

We can use the get_entity_vals functon from the mne_bids package to find all the subjects in the database. Once we have selected a subject, we can use the same function to find all sessions and experiments (called 'task' in OpenBIDS format) associated with that subject. Finally, we can find the .tsv file containing the behavioral data and load it into a Pandas Dataframe.

In [6]:
# set root
bids_root = "/home1/maint/LTP_BIDS/"

In [7]:
# let's find subjects loaded in the database
subjects = get_entity_vals(bids_root, "subject")
subjects

['LTP063',
 'LTP064',
 'LTP065',
 'LTP066',
 'LTP067',
 'LTP068',
 'LTP069',
 'LTP070',
 'LTP073',
 'LTP074',
 'LTP093',
 'LTP106',
 'LTP115',
 'LTP117',
 'LTP122',
 'LTP123',
 'LTP133',
 'LTP138',
 'LTP187',
 'LTP207',
 'LTP229',
 'LTP246',
 'LTP260',
 'LTP312',
 'LTP327',
 'LTP329',
 'LTP339',
 'LTP347',
 'LTP354',
 'LTP355',
 'LTP606',
 'LTP607',
 'LTP609',
 'LTP610',
 'LTP612',
 'LTP613',
 'LTP614']

In [13]:
# now get the root of the subject and find which experiments they have completed
subject_root = os.path.join(bids_root, f"sub-LTP606")
tasks = get_entity_vals(subject_root, "task")
tasks

['valuecourier']

In [14]:
# we can also see how many sessions they have completed
sessions = get_entity_vals(subject_root, "session")
sessions

['0', '1', '2', '3', '4', '5']

In [20]:
# plug in the subject, task, and session info into BIDSPath and have the datatype set to "beh" to get the path to the .tsv file
# all inputs to BIDSPath should be strings so convert using the str() function
# load it using read_csv
subject = "LTP606"
task = tasks[0]
session = sessions[0]

path_bids = BIDSPath(
                subject=subject,
                session=str(session),
                task=task,  
                datatype="beh", 
                suffix="beh",
                extension=".tsv",
                root=bids_root
            )
evs_bids = pd.read_csv(path_bids.fpath, sep="\t")
evs_bids[:10]
evs_bids.columns

Index(['mstime', 'trial_type', 'stim_file', 'actualvalue', 'compensation',
       'eegfile', 'eogArtifact', 'experiment', 'intruded', 'intrusion', 'item',
       'itemno', 'itemvalue', 'montage', 'msoffset', 'multiplier',
       'numingroupchosen', 'phase', 'playerrotY', 'presX', 'presZ',
       'primacybuf', 'protocol', 'recalled', 'rectime', 'recencybuf',
       'serialpos', 'session', 'store', 'storeX', 'storeZ', 'storepointtype',
       'subject', 'trial', 'valuerecall'],
      dtype='object')

In [11]:
# we can also set the datatype to "eeg" and load it this way
# by setting it to "eeg", we avoid having to remake a BIDSPath object for both the EEG and behavioral data
bids_path = BIDSPath(
    subject=subject,
    session=str(session),
    task=task,
    datatype="eeg",
    root=bids_root,
)

events_path = os.path.join(bids_path.directory, bids_path.basename + "_events.tsv")
evs_bids = pd.read_csv(events_path, sep="\t")
evs_bids[:10]

FileNotFoundError: [Errno 2] No such file or directory: '/home1/maint/LTP_BIDS/sub-LTP063/ses-0/eeg/sub-LTP063_ses-0_task-ltpfr_events.tsv'

## Differences Between CML and OpenBIDS Events

The event dataframes loaded via OpenBIDS and CMLReader are identical in content, with the only differences being a small number of additional or renamed columns in the OpenBIDS version that conform to OpenBIDS formatting standards.

In [ ]:
evs_cml.columns

In [ ]:
evs_bids.columns

### Column differences
| CMLReader column | OpenBIDS column | Notes |
|------------------|-----------------|-------|
| `eegoffset` | `sample` | Same information; renamed to match BIDS conventions |
| `type` | `trial_type` | Renamed for BIDS compliance |
| `mstime` | `mstime` | In BIDS, `mstime` is typically shifted so that `mstime[0] = 0` |
| *(derived)* | `onset` | Computed as `sample / sample_frequency` |
| *(not present)* | `duration` | Not used / not applicable |
| *(not present)* | `stim_file` | Empty string placeholder required by BIDS |
                          

## Loading Raw EEG

Here's how we load the whole raw EEG file without creating epochs.

### CMLReader

In [ ]:
# use reader from events
eeg_cml = cmlreader.load_eeg().to_ptsa()

### OpenBIDS

In [ ]:
# use BIDSPath with datatype set to "eeg"
bids_path = BIDSPath(
            subject=subject,
            session=str(session),
            task=task,
            datatype="eeg",
            root=bids_root,
        )

# load the header from the bids file
raw = read_raw_bids(
    bids_path,
    verbose=True,
)

In [ ]:
# ptsa is just a wrapper for XArray, so we can just create an XArray object using the data from raw
eeg_bids = xr.DataArray(
    raw.get_data()[None, :, :],                # load the raw eeg data                  
    dims=("event", "channel", "time"),      
    coords={
        "event": [0],                          # singleton event index
        "channel": raw.ch_names,
        "time": raw.times  ,
        "samplerate": raw.info["sfreq"],          
    },
    name="eeg",
)
eeg_bids

## Loading Epoched EEG

Here's how to load Epoched EEG data. First let's set the constants.

In [ ]:
# constants
REL_START, REL_STOP = 200, 3000
BUFFER_MS = 1000
WIDTH = 6

FREQS = np.logspace(np.log10(2), np.log10(100), 46)
NOTCH_BAND = (58., 62.)
BATCH_EVENTS = 64

### CMLReader

It is easy to load the eeg data using the load_eeg method of the Reader object.

In [ ]:
# many of our analyses use the clean function, but to get the same data as the BIDS loader, we will forego it
# epochs_cml = reader.load_eeg(
#     word_evs_cml, rel_start=-BUFFER_MS, rel_stop=REL_STOP + BUFFER_MS, clean='LCF'
# ).to_ptsa()

# filter events
word_evs_cml = evs_cml[evs_cml["type"] == "WORD"]

epochs_cml = cmlreader.load_eeg(
    word_evs_cml, rel_start=-BUFFER_MS, rel_stop=REL_STOP + BUFFER_MS
).to_ptsa()
epochs_cml

### OpenBIDS

We can load epochs for OpenBIDS using the MNE Epochs object then converting to ptsa.

REMINDER: MNE Epochs uses seconds so we must divide by 1000 if the constants were in seconds.

In [ ]:
# get BIDSPath
bids_path = BIDSPath(
    subject=subject,
    session=str(session),
    task=task,
    datatype="eeg",
    root=bids_root,
)

# and load raw data
raw = read_raw_bids(bids_path)

# set non-EEG channels
raw.set_channel_types({
    "EXG1": "eog", "EXG2": "eog", "EXG3": "eog", "EXG4": "eog",
    "EXG5": "misc", "EXG6": "misc", "EXG7": "misc", "EXG8": "misc",
})

### Filtering Events for Loading

In order to select the events that we will epoch over, we must get the string representation from the events from the raw data's header using the events_from_annotations function. The function returns "events" which maps the sample to the event index and "event_id" which maps each event name to its index.

In [ ]:
# get events from raw data's header
events, event_id = mne.events_from_annotations(raw)

To filter the events, we select only the event_ids we are interested in.

In [ ]:
# filter word events and return the index of the WORD event
word_event_id = {k: v for k, v in event_id.items() if k == "WORD"}
word_event_id

In [ ]:
tmin = (-BUFFER_MS / 1000)
tmax = ((REL_STOP /1000 + BUFFER_MS / 1000))

# load into MNE Epochs object
epochs_mne = mne.Epochs(
    raw,
    events=events,                      # we can now load the events here
    event_id=word_event_id,             # we cna also load the filtered events here
    tmin=tmin,
    tmax=tmax,
    baseline=(None, 0), # None goes to start, 0 is onset of stimuli
    preload=True,
    event_repeated="merge",
)
epochs_mne

In [ ]:
# convert to ptsa
epochs_bids = TimeSeries.from_mne_epochs(epochs_mne, word_evs_bids)
epochs_bids